In [3]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [5]:
# read in with pandas
winter1 = pd.read_csv ('sfz20-21data.txt')
winter2 = pd.read_csv ('sfz21-22data.txt')
winter3 = pd.read_csv ('sfz22-23data.txt')

In [7]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [9]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [12]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [14]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [16]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [18]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [20]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [22]:
sfz = LocationInfo(
    'Pawtucket', 'Rhode Island', 
    latitude=41.8787,  
    longitude=-71.3826,
    timezone='America/Rhode_Island'
)

sfz

LocationInfo(name='Pawtucket', region='Rhode Island', timezone='America/Rhode_Island', latitude=41.8787, longitude=-71.3826)

In [24]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(sfz.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [26]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(sfz.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [28]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(sfz.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [30]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:54:24  2020-12-01 16:14:57  2020-12-01
1   2020-12-02 06:55:26  2020-12-02 16:14:42  2020-12-02
2   2020-12-03 06:56:27  2020-12-03 16:14:30  2020-12-03
3   2020-12-04 06:57:27  2020-12-04 16:14:20  2020-12-04
4   2020-12-05 06:58:25  2020-12-05 16:14:13  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:28:04  2021-02-24 17:29:52  2021-02-24
86  2021-02-25 06:26:32  2021-02-25 17:31:05  2021-02-25
87  2021-02-26 06:24:59  2021-02-26 17:32:18  2021-02-26
88  2021-02-27 06:23:26  2021-02-27 17:33:30  2021-02-27
89  2021-02-28 06:21:51  2021-02-28 17:34:42  2021-02-28

[90 rows x 3 columns]


In [32]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:54:09  2021-12-01 16:15:01  2021-12-01
1   2021-12-02 06:55:11  2021-12-02 16:14:45  2021-12-02
2   2021-12-03 06:56:13  2021-12-03 16:14:33  2021-12-03
3   2021-12-04 06:57:13  2021-12-04 16:14:22  2021-12-04
4   2021-12-05 06:58:11  2021-12-05 16:14:14  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:28:26  2022-02-24 17:29:35  2022-02-24
86  2022-02-25 06:26:54  2022-02-25 17:30:48  2022-02-25
87  2022-02-26 06:25:22  2022-02-26 17:32:00  2022-02-26
88  2022-02-27 06:23:48  2022-02-27 17:33:12  2022-02-27
89  2022-02-28 06:22:14  2022-02-28 17:34:25  2022-02-28

[90 rows x 3 columns]


In [34]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:53:54  2022-12-01 16:15:05  2022-12-01
1   2022-12-02 06:54:56  2022-12-02 16:14:49  2022-12-02
2   2022-12-03 06:55:58  2022-12-03 16:14:35  2022-12-03
3   2022-12-04 06:56:58  2022-12-04 16:14:24  2022-12-04
4   2022-12-05 06:57:57  2022-12-05 16:14:15  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:28:48  2023-02-24 17:29:17  2023-02-24
86  2023-02-25 06:27:16  2023-02-25 17:30:30  2023-02-25
87  2023-02-26 06:25:44  2023-02-26 17:31:43  2023-02-26
88  2023-02-27 06:24:11  2023-02-27 17:32:55  2023-02-27
89  2023-02-28 06:22:37  2023-02-28 17:34:07  2023-02-28

[90 rows x 3 columns]


In [36]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [38]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,SFZ,2020-12-01 00:56:00,7.87,-RA BR,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
1,SFZ,2020-12-01 01:56:00,5.59,-RA,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
2,SFZ,2020-12-01 02:56:00,2.29,-RA,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
3,SFZ,2020-12-01 03:56:00,3.30,-RA BR,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
4,SFZ,2020-12-01 04:56:00,4.32,-RA BR,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
...,...,...,...,...,...,...,...
315,SFZ,2021-02-27 16:56:00,1.27,-RA BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30
316,SFZ,2021-02-27 17:56:00,2.54,RA BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30
317,SFZ,2021-02-27 18:56:00,2.29,-RA BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30
318,SFZ,2021-02-27 19:56:00,2.03,-RA BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30


In [40]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,SFZ,2021-12-02 09:56:00,0.0000,-RA,2021-12-02,2021-12-02 06:55:11,2021-12-02 16:14:45
1,SFZ,2021-12-02 11:56:00,0.0000,-RA,2021-12-02,2021-12-02 06:55:11,2021-12-02 16:14:45
2,SFZ,2021-12-02 12:56:00,0.0000,-RA,2021-12-02,2021-12-02 06:55:11,2021-12-02 16:14:45
3,SFZ,2021-12-03 03:56:00,0.0000,-RA,2021-12-03,2021-12-03 06:56:13,2021-12-03 16:14:33
4,SFZ,2021-12-06 10:56:00,0.0000,RA BR,2021-12-06,2021-12-06 06:59:09,2021-12-06 16:14:08
...,...,...,...,...,...,...,...
367,SFZ,2022-02-25 19:56:00,0.5100,-SN BR,2022-02-25,2022-02-25 06:26:54,2022-02-25 17:30:48
368,SFZ,2022-02-25 20:56:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:26:54,2022-02-25 17:30:48
369,SFZ,2022-02-25 21:56:00,0.0001,-SN,2022-02-25,2022-02-25 06:26:54,2022-02-25 17:30:48
370,SFZ,2022-02-25 22:56:00,0.0001,-SN,2022-02-25,2022-02-25 06:26:54,2022-02-25 17:30:48


In [42]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,SFZ,2022-12-01 00:56:00,6.3500,-RA,2022-12-01,2022-12-01 06:53:54,2022-12-01 16:15:05
1,SFZ,2022-12-01 01:56:00,1.0200,-RA,2022-12-01,2022-12-01 06:53:54,2022-12-01 16:15:05
2,SFZ,2022-12-03 15:56:00,0.0001,-RA,2022-12-03,2022-12-03 06:55:58,2022-12-03 16:14:35
3,SFZ,2022-12-03 16:56:00,0.5100,-RA,2022-12-03,2022-12-03 06:55:58,2022-12-03 16:14:35
4,SFZ,2022-12-03 17:56:00,2.2900,-RA BR,2022-12-03,2022-12-03 06:55:58,2022-12-03 16:14:35
...,...,...,...,...,...,...,...
361,SFZ,2023-02-26 01:56:00,0.0001,-SN,2023-02-26,2023-02-26 06:25:44,2023-02-26 17:31:43
362,SFZ,2023-02-26 02:56:00,0.0001,-SN,2023-02-26,2023-02-26 06:25:44,2023-02-26 17:31:43
363,SFZ,2023-02-26 05:56:00,0.0001,-SN,2023-02-26,2023-02-26 06:25:44,2023-02-26 17:31:43
364,SFZ,2023-02-27 09:56:00,0.0001,-SN,2023-02-27,2023-02-27 06:24:11,2023-02-27 17:32:55


In [44]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
12,SFZ,2020-12-05 07:56:00,0.2500,-RA,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13
13,SFZ,2020-12-05 08:56:00,0.2500,-RA,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13
14,SFZ,2020-12-05 09:56:00,1.0200,-RA BR,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13
15,SFZ,2020-12-05 10:56:00,2.0300,-RA BR,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13
16,SFZ,2020-12-05 11:56:00,2.2900,-RA,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13
...,...,...,...,...,...,...,...
311,SFZ,2021-02-27 12:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30
312,SFZ,2021-02-27 13:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30
313,SFZ,2021-02-27 14:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30
314,SFZ,2021-02-27 15:56:00,0.2500,-RA BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30


In [46]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [48]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
13,SFZ,2020-12-05 08:56:00,0.2500,-RA,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13,2020-12-05 07:58:25,2020-12-05 15:14:13
14,SFZ,2020-12-05 09:56:00,1.0200,-RA BR,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13,2020-12-05 07:58:25,2020-12-05 15:14:13
15,SFZ,2020-12-05 10:56:00,2.0300,-RA BR,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13,2020-12-05 07:58:25,2020-12-05 15:14:13
16,SFZ,2020-12-05 11:56:00,2.2900,-RA,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13,2020-12-05 07:58:25,2020-12-05 15:14:13
17,SFZ,2020-12-05 12:56:00,2.2900,-RA BR,2020-12-05,2020-12-05 06:58:25,2020-12-05 16:14:13,2020-12-05 07:58:25,2020-12-05 15:14:13
...,...,...,...,...,...,...,...,...,...
310,SFZ,2021-02-27 11:56:00,0.0001,UP BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30,2021-02-27 07:23:26,2021-02-27 16:33:30
311,SFZ,2021-02-27 12:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30,2021-02-27 07:23:26,2021-02-27 16:33:30
312,SFZ,2021-02-27 13:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30,2021-02-27 07:23:26,2021-02-27 16:33:30
313,SFZ,2021-02-27 14:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30,2021-02-27 07:23:26,2021-02-27 16:33:30


In [50]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [52]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [54]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.13291714285714293
2021-2022: 0.6347389705882351
2022-2023: 0.8014205882352942


In [56]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,SFZ,2020-12-01 00:56:00,7.8700,-RA BR,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
1,SFZ,2020-12-01 01:56:00,5.5900,-RA,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
2,SFZ,2020-12-01 02:56:00,2.2900,-RA,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
3,SFZ,2020-12-01 03:56:00,3.3000,-RA BR,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
4,SFZ,2020-12-01 04:56:00,4.3200,-RA BR,2020-12-01,2020-12-01 06:54:24,2020-12-01 16:14:57
...,...,...,...,...,...,...,...
308,SFZ,2021-02-25 05:56:00,0.0001,-RA,2021-02-25,2021-02-25 06:26:32,2021-02-25 17:31:05
316,SFZ,2021-02-27 17:56:00,2.5400,RA BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30
317,SFZ,2021-02-27 18:56:00,2.2900,-RA BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30
318,SFZ,2021-02-27 19:56:00,2.0300,-RA BR,2021-02-27,2021-02-27 06:23:26,2021-02-27 17:33:30


In [58]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22028\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [60]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [62]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [64]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.3053479591836735
2021-2022: 0.16609876543209875
2022-2023: 0.646181188118812
